<a href="https://colab.research.google.com/github/GiedriusDapsys/MB-Ailena-Anomaly-Detection/blob/main/notebooks/MB%20Ailena%20Anomaly%20Detection%20Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

DATA_PATH = "/content/Pardavimai prekiu grupes  2025 01 01 06 30.xlsx"  # pritaikyk tikslų vardą
df = pd.read_excel(DATA_PATH, skiprows=5, header=0)
df.head()


In [ ]:
# ---- 1. VALYMAS ----
# Užpildom grupių pavadinimus (aukščiau buvę antraščių blokai sukelia NaN)
df['Pasirinkta prekė(Grupė)'] = df['Pasirinkta prekė(Grupė)'].ffill()

# Paverčiam mėnesį į datetime
df['Mėnuo'] = pd.to_datetime(df['Mėnuo'].astype(str), errors='coerce')

# Skaitiniai stulpeliai -> float ir užpildom spragas
num_cols = ['Kiekis','Pirkimo suma EUR','Pardavimo suma EUR','Pelnas']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce').fillna(method='ffill').fillna(method='bfill')

# Išmetam aiškius meta-įrašus (jei tokių būtų)
mask_meta = (df['Pasirinkta prekė(Grupė)'].astype(str).str.contains('Iš viso|VISO', case=False, na=False)) | (df['Mėnuo'].isna())
df = df[~mask_meta].reset_index(drop=True)

print(df.dtypes)
df.head(3)


In [ ]:
import numpy as np

window_size = 12             # gali keisti
X = df[num_cols].values      # 4 bruožai

# suformuojam sekas: [N-w+1, w, features]
seqs = np.array([X[i:i+window_size] for i in range(len(X)-window_size+1)])
print("seqs shape:", seqs.shape)

# paprastas 80/20 split
split = int(len(seqs)*0.8)
train_seqs, test_seqs = seqs[:split], seqs[split:]
train_seqs.shape, test_seqs.shape


In [ ]:
from keras import layers, models
import matplotlib.pyplot as plt

timesteps, n_features = window_size, train_seqs.shape[2]
latent_dim = 8  # galima eksperimentuoti

inp = layers.Input(shape=(timesteps, n_features))
enc = layers.LSTM(latent_dim, activation='tanh')(inp)
rep = layers.RepeatVector(timesteps)(enc)
dec = layers.LSTM(n_features, activation='tanh', return_sequences=True)(rep)

autoencoder = models.Model(inp, dec)
autoencoder.compile(optimizer='adam', loss='mse')
history = autoencoder.fit(
    train_seqs, train_seqs,
    epochs=20, batch_size=16, validation_split=0.1, verbose=1
)

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend(); plt.title('Autoencoder Loss'); plt.show()


In [ ]:
# rekonstrukcijos klaida (MSE) test rinkinyje
recon = autoencoder.predict(test_seqs, verbose=0)
mse = np.mean((test_seqs - recon)**2, axis=(1,2))

pct = 85  # gali keisti (70–95)
thr = np.percentile(mse, pct)
flags = mse > thr
print(f"Threshold @{pct}th: {thr:.6f} | anomalies: {flags.sum()}/{len(flags)}")

# žemėlapiavimas atgal į mėnesius ir grupes
months = df['Mėnuo'].dt.to_period('M').astype(str)
groups = df['Pasirinkta prekė(Grupė)'].astype(str)

rows = []
for i, is_anom in enumerate(flags):
    if not is_anom:
        continue
    idx = split + i + window_size - 1   # indekso perslinkimas
    m = months.iloc[idx]
    g = groups.iloc[idx]
    # atmetam „Iš viso“ ir kt. triukšmą, jei patektų
    if m != 'NaT' and g and ':' not in g and not g.lower().startswith('iš viso'):
        rows.append({'Grupė': g, 'Mėnuo': m})

import pandas as pd
anom_df = pd.DataFrame(rows)
anom_df.head()


In [ ]:
# Aggregation: kiek anomalijų per grupę ir mėnesį
pivot_anom = anom_df.pivot_table(index='Grupė', columns='Mėnuo', aggfunc='size', fill_value=0)
pivot_anom['Iš viso'] = pivot_anom.sum(axis=1)
pivot_anom = pivot_anom.sort_values('Iš viso', ascending=False)

display(pivot_anom.head(15))

# TOP-10 barh
top10 = pivot_anom.head(10)
plt.figure(figsize=(8,5))
plt.barh(top10.index, top10['Iš viso'], color='tomato')
plt.gca().invert_yaxis()
plt.xlabel('Anomalijų skaičius'); plt.title('TOP-10 prekių grupės pagal anomalijas')
plt.tight_layout(); plt.show()


In [ ]:
percentiles = [70, 75, 80, 85, 90, 95]
counts = []
for p in percentiles:
    thr = np.percentile(mse, p)
    counts.append((flags := (mse > thr)).sum())

plt.figure(figsize=(6,4))
plt.plot(percentiles, counts, marker='o')
plt.xlabel('Percentilės slenkstis'); plt.ylabel('Anomalijų skaičius')
plt.title('Jautrumo analizė'); plt.grid(True, alpha=.3); plt.show()


In [ ]:
# paimam top3 pagal 'Iš viso'
top3 = list(pivot_anom.head(3).index)

fig, axes = plt.subplots(len(top3), 1, figsize=(10, 3*len(top3)), sharex=True)
if len(top3) == 1: axes = [axes]

for ax, grp in zip(axes, top3):
    sub = df[df['Pasirinkta prekė(Grupė)'] == grp].copy()
    sub = sub.sort_values('Mėnuo')
    ax.plot(sub['Mėnuo'], sub['Pardavimo suma EUR'], label='Pardavimo suma EUR')

    # pažymim anomalijų mėnesius šiai grupei
    months_anom = anom_df.loc[anom_df['Grupė'] == grp, 'Mėnuo'].unique()
    mark = sub['Mėnuo'].dt.to_period('M').astype(str).isin(months_anom)
    ax.scatter(sub.loc[mark, 'Mėnuo'], sub.loc[mark, 'Pardavimo suma EUR'], color='red', zorder=3, label='Anomalija')

    ax.set_title(grp); ax.legend()

plt.tight_layout(); plt.show()


In [ ]:
import numpy as np

# Flatten: (n_samples, window_size * n_features)
train_flat = train_seqs.reshape(train_seqs.shape[0], -1)
test_flat  = test_seqs.reshape(test_seqs.shape[0],  -1)

train_flat.shape, test_flat.shape


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

# 1) Isolation Forest
iforest = IsolationForest(
    n_estimators=200, contamination='auto', random_state=42
)
iforest.fit(train_flat)
s_if = -iforest.decision_function(test_flat)   # didesnis -> labiau anomalija

# 2) One-Class SVM (RBF)
ocsvm = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale')
ocsvm.fit(train_flat)
s_svm = -ocsvm.decision_function(test_flat)    # invertuojam, kad didesnis -> anomalija

# 3) Local Outlier Factor (novelty=True kad tiktų .predict ant naujų)
lof = LocalOutlierFactor(n_neighbors=35, novelty=True, contamination='auto')
lof.fit(train_flat)
s_lof = -lof.decision_function(test_flat)      # didesnis -> anomalija


In [ ]:
import pandas as pd

def minmax(x):
    x = np.asarray(x, float)
    return (x - x.min()) / (x.max() - x.min() + 1e-12)

scores = {
    'AE_MSE': minmax(mse),    # iš jūsų AE
    'IForest': minmax(s_if),
    'OCSVM': minmax(s_svm),
    'LOF': minmax(s_lof),
}

pct = 85  # tas pats kaip AE
thresholds = {k: np.percentile(v, pct) for k, v in scores.items()}
flags = {k: (v > thresholds[k]) for k, v in scores.items()}

# Bendras palyginimo DF (test langų lygiu)
comp = pd.DataFrame(scores)
comp['month'] = months.iloc[split + window_size - 1 : split + window_size - 1 + len(comp)].values
comp['group'] = groups.iloc[split + window_size - 1 : split + window_size - 1 + len(comp)].values
for k in flags:
    comp[f'{k}_flag'] = flags[k].astype(int)

display(comp.head())


In [ ]:
def jaccard(a, b):
    a = set(np.where(a)[0]); b = set(np.where(b)[0])
    return len(a & b) / len(a | b) if len(a | b) > 0 else 0.0

base = 'AE_MSE'
rows = []
for k in scores:
    n = int(flags[k].sum())
    jac = 1.0 if k == base else jaccard(flags[base], flags[k])
    rows.append([k, n, thresholds[k], jac])

summary_df = pd.DataFrame(rows, columns=['Model', 'AnomalyCount', 'Threshold', 'Jaccard_vs_AE'])
display(summary_df)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.bar(summary_df['Model'], summary_df['AnomalyCount'])
plt.title(f'Anomalijų skaičius (@{pct}th)'); plt.ylabel('Count'); plt.show()


In [ ]:
def anomalies_to_pivot(model_key):
    mask = flags[model_key]
    rows = []
    for i, is_an in enumerate(mask):
        if not is_an:
            continue
        idx = split + i + window_size - 1
        m = months.iloc[idx]; g = groups.iloc[idx]
        # atmetam bendras sumas / meta eiles
        if m != 'NaT' and isinstance(g, str) and ':' not in g and not g.lower().startswith('iš viso'):
            rows.append((g, m))
    adf = pd.DataFrame(rows, columns=['Grupė','Mėnuo'])
    if adf.empty:
        return pd.DataFrame()
    p = adf.pivot_table(index='Grupė', columns='Mėnuo', aggfunc='size', fill_value=0)
    p['Iš viso'] = p.sum(axis=1)
    return p.sort_values('Iš viso', ascending=False)

pivot_ae = anomalies_to_pivot('AE_MSE').head(10)
pivot_if = anomalies_to_pivot('IForest').head(10)
pivot_svm = anomalies_to_pivot('OCSVM').head(10)
pivot_lof = anomalies_to_pivot('LOF').head(10)

print("AE – TOP10"); display(pivot_ae)
print("IForest – TOP10"); display(pivot_if)
print("OCSVM – TOP10"); display(pivot_svm)
print("LOF – TOP10"); display(pivot_lof)


In [ ]:
# === IŠSAUGOJIMAS: grafikai ir lentelės -> /content ===
import os
os.makedirs("/content/report/img", exist_ok=True)
os.makedirs("/content/report/tables", exist_ok=True)

# 1) Modelių anomalijų skaičiaus grafikas
plt.figure(figsize=(6,4))
plt.bar(summary_df['Model'], summary_df['AnomalyCount'])
plt.title(f'Anomalijų skaičius (@{pct}th)'); plt.ylabel('Count')
plt.tight_layout()
plt.savefig("/content/report/img/anomaly_count_by_model.png", dpi=160, bbox_inches='tight')
plt.show()

# 2) Modelių palyginimo lentelė (CSV ir Markdown)
summary_df.to_csv("/content/report/tables/model_summary.csv", index=False)
with open("/content/report/tables/model_summary.md", "w", encoding="utf-8") as f:
    f.write(summary_df.to_markdown(index=False))

# 3) TOP-10 pivot lentelės pagal modelį (CSV ir Markdown)
def save_pivot(name, pivot):
    if pivot is None or pivot.empty:
        return
    pivot10 = pivot.head(10)
    pivot10.to_csv(f"/content/report/tables/{name}_top10.csv")
    with open(f"/content/report/tables/{name}_top10.md", "w", encoding="utf-8") as f:
        f.write(pivot10.to_markdown())

save_pivot("ae",    pivot_ae)
save_pivot("ifor",  pivot_if)
save_pivot("ocsvm", pivot_svm)
save_pivot("lof",   pivot_lof)

print("✓ Išsaugota į /content/report/img ir /content/report/tables")


In [ ]:
!zip -r /content/report_artifacts.zip /content/report
from google.colab import files
files.download('/content/report_artifacts.zip')


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path('/content/report/tables')  # jei reikia, pataisyk kelią

# 1) Įkeliam TOP lenteles (jei .md – naudok read_table; jei .csv – read_csv)
def read_any(p):
    p = Path(p)
    if p.suffix == '.csv':
        return pd.read_csv(p)
    elif p.suffix == '.md':
        # md konvertuojam į csv-like, praleidžiam header separatorius
        df = pd.read_table(p, sep='|', engine='python', skiprows=1)
        # numest nereikalingas kolonas (md turi daug tuščių stulpelių)
        df = df.dropna(axis=1, how='all')
        # išvalom whitespace ir header eilutę, jei yra
        df.columns = [c.strip() for c in df.columns]
        # kartais pirmas/galinis stulpelis būna tuščias
        keep = [c for c in df.columns if c not in ['', 'Unnamed: 0']]
        df = df[keep]
        return df
    else:
        raise ValueError(f'Nežinomas formatas: {p}')

ae   = read_any(BASE/'ae_top10.csv')
ifor = read_any(BASE/'ifor_top10.csv')
ocsvm= read_any(BASE/'ocsvm_top10.csv')
lof  = read_any(BASE/'lof_top10.csv')

# 2) Sutvarkom pavadinimus: bandysim atspėti laukus
def normalize_cols(df):
    cols = {c.lower(): c for c in df.columns}
    # bandome atspėti stulpelių pavadinimus
    group_col = next((cols[k] for k in cols if 'grup' in k or 'prek' in k), None)
    month_col = next((cols[k] for k in cols if 'mėnuo' in k or 'menuo' in k or 'men' in k), None)
    score_col = next((cols[k] for k in cols if 'score' in k or 'mse' in k or 'klaida' in k), None)

    # paliekam tik naudingus
    keep = [c for c in [group_col, month_col, score_col] if c is not None]
    out = df[keep].copy()
    # pervadinam
    ren = {}
    if group_col: ren[group_col] = 'Group'
    if month_col: ren[month_col] = 'Month'
    if score_col: ren[score_col] = 'Score'
    out = out.rename(columns=ren)
    return out

ae   = normalize_cols(ae)
ifor = normalize_cols(ifor)
ocsvm= normalize_cols(ocsvm)
lof  = normalize_cols(lof)

# 3) „Raktas“ – grupė + mėnuo (kas apibrėžia vieną anomalijos langą)
for df in [ae, ifor, ocsvm, lof]:
    if 'Month' in df.columns:
        df['Month'] = pd.to_datetime(df['Month'], errors='coerce')

def key_df(df, model_name):
    if 'Group' in df.columns and 'Month' in df.columns:
        k = df[['Group','Month']].dropna().copy()
        k['model'] = model_name
        return k
    elif 'Group' in df.columns:
        k = df[['Group']].dropna().copy()
        k['Month'] = pd.NaT
        k['model'] = model_name
        return k
    else:
        return pd.DataFrame(columns=['Group','Month','model'])

K = pd.concat([
    key_df(ae, 'AE_MSE'),
    key_df(ifor, 'IForest'),
    key_df(ocsvm, 'OCSVM'),
    key_df(lof, 'LOF')
], ignore_index=True).drop_duplicates()

# 4) Skaičiai: kiek per modelį
per_model_counts = K.groupby('model').size().sort_index()
print('Anomalijų per modelį:\n', per_model_counts, '\n')

# 5) Unikalus anomalijų langų skaičius (sąjunga tarp modelių)
unique_windows = K[['Group','Month']].drop_duplicates().shape[0]
print('Bendras unikalių anomalijų (grupė+mėnuo) skaičius:', unique_windows, '\n')

# 6) Kurios grupės dažniausiai pasikartoja (kartu tarp modelių)
grp_freq = K.groupby('Group').size().sort_values(ascending=False)
print('Grupių dažniai (kartų skaičius tarp modelių):\n', grp_freq.head(10), '\n')

# 7) Persidengimai: kiek langų sutampa tarp modelių (Jaccard rodyklė tarp porų)
def jaccard(A, B):
    inter = len(A & B)
    union = len(A | B)
    return inter/union if union>0 else 0.0

def keyset(df):
    return set(zip(df['Group'], df['Month']))

S = {
    'AE_MSE': keyset(key_df(ae,'AE_MSE')),
    'IForest': keyset(key_df(ifor,'IForest')),
    'OCSVM' : keyset(key_df(ocsvm,'OCSVM')),
    'LOF'   : keyset(key_df(lof,'LOF')),
}

pairs = [('AE_MSE','IForest'), ('AE_MSE','OCSVM'), ('AE_MSE','LOF'),
         ('IForest','OCSVM'), ('IForest','LOF'), ('OCSVM','LOF')]

print('Poriniai persidengimai (Jaccard):')
for a,b in pairs:
    print(f'  {a} vs {b}: {jaccard(S[a], S[b]):.2f}')


In [ ]:
# ==== Jautrumo kreivė: automatinis Excel paėmimas + braižymas =================
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import IsolationForest

# (1) Jei žinai tikslų kelią – įrašyk čia ir atkomentuok:
# DATA_PATH = "/content/Pardavimai prekiu grupes  2025 01 01 06 30.xlsx"

def ensure_excel_path():
    """Grąžina Excel failo kelią: iš DATA_PATH, /content paieškos arba paprašant įkelti."""
    # a) jei DATA_PATH jau yra apibrėžtas ir egzistuoja
    if 'DATA_PATH' in globals():
        p = Path(DATA_PATH)
        if p.exists():
            return str(p)

    # b) ieškome /content *.xlsx
    candidates = glob.glob("/content/*.xlsx")
    if candidates:
        # jei yra keli, pirmenybė su 'Pardav' pavadinime, kitaip pirmas
        for p in candidates:
            if "Pardav" in os.path.basename(p):
                return p
        return candidates[0]

    # c) paprašome įkelti (Colab failų įkėlimo langas)
    try:
        from google.colab import files
        uploaded = files.upload()  # vartotojas pasirenka .xlsx
        if uploaded:
            # imame pirmą įkeltą
            fname = list(uploaded.keys())[0]
            return f"/content/{fname}"
    except Exception as e:
        print("Nepavyko paleisti files.upload(). Klaida:", e)

    raise FileNotFoundError(
        "Nerasta .xlsx. Įkelkite Excel į Colab (kairėje Files skydelyje Upload) "
        "arba nurodykite DATA_PATH = '/content/tavo_failas.xlsx'."
    )

# 1) Užtikrinam kelią ir įkeliame duomenis į df
excel_path = ensure_excel_path()
print(f"Naudojamas Excel: {excel_path}")
df = pd.read_excel(excel_path, skiprows=5, header=0)

# 2) Pasirenkame skaitinius stulpelius (kiekis/pirkimo/pardavimo/pelnas) – jautrumui užteks kelių
def pick_col(df, candidates):
    low = {c.lower(): c for c in df.columns}
    for cand in candidates:
        for k, orig in low.items():
            if cand in k:
                return orig
    return None

col_kiekis = pick_col(df, ['kiekis'])
col_buy    = pick_col(df, ['pirkimo suma', 'pirkimo'])
col_sell   = pick_col(df, ['pardavimo suma', 'pardavimo'])
col_profit = pick_col(df, ['pelnas'])

use_cols = [c for c in [col_kiekis, col_buy, col_sell, col_profit] if c is not None]
if len(use_cols) < 2:
    raise ValueError(f"Neradau pakankamai skaitinių stulpelių. Rasta: {use_cols}")

X = df[use_cols].apply(pd.to_numeric, errors='coerce')
X = X.fillna(X.median())

# 3) Isolation Forest balas (kuo didesnis, tuo 'keistesnis')
iso = IsolationForest(n_estimators=300, contamination=0.05, random_state=42)
iso.fit(X)
scores = -iso.score_samples(X)

# 4) Jautrumo kreivė – kiek anomalijų, jei slenkstis p-tasis procentilis
pct_list = [70, 75, 80, 85, 90, 95]
rows = []
for p in pct_list:
    thr = np.percentile(scores, p)
    n_anom = int((scores > thr).sum())
    rows.append({'pct': p, 'n': n_anom})
sensitivity_df = pd.DataFrame(rows)
print("sensitivity_df:\n", sensitivity_df)

# 5) Išsaugome grafiką
out_dir = Path('/content/report/img')
out_dir.mkdir(parents=True, exist_ok=True)

plt.figure()
ax = sensitivity_df.set_index('pct')['n'].plot(marker='o')
ax.set_xlabel('Percentile threshold')
ax.set_ylabel('Number of anomalies')
ax.set_title('Jautrumo kreivė (IsolationForest)')
plt.tight_layout()
out_path = out_dir / 'sensitivity.png'
plt.savefig(out_path, dpi=150)
plt.close()

print(f"✓ Grafikas išsaugotas: {out_path}")
